Extract scaffold 12 from assembly.

In [1]:
from Bio import SeqIO
import gzip

target = 'scaffold_12'

assembly = gzip.open('../../data/REFERENCE_DATA/M_zebra_v0.assembly.fasta.gz')

for rec in SeqIO.parse(assembly, 'fasta'):
    if rec.id == target:
        out = open(target+'.fasta','w')
        SeqIO.write(rec, out, 'fasta')
        out.close()

Extract full length CDS according to gff file for target gene.

Get coordinates/orientation for rhodopsin gene exons from gtf file. Gene id is: `s12.95`.

In [2]:
!zcat ../../data/REFERENCE_DATA/Metriaclima_zebra.BROADMZ2.gtf.gz | grep "s12.95\"" | grep "CDS" | cut -f 4,5,7

3793413	3794477	-


In [3]:
!zcat ../../data/REFERENCE_DATA/Metriaclima_zebra.BROADMZ2.gtf.gz | grep "s12.95\"" | grep "CDS" | cut -f 4,5 > target_exons.tsv

In [4]:
%%bash

vcf='../../data/VCF/Malawi2015_all_sc_12_3750000-3850000_beagle_annotated.vcf.gz'

python ../../../../Dropbox/Github/genomisc/popogeno/vcf_2_hap.py $vcf \
--fasta \
--fill_from_ref_fasta scaffold_12.fasta \
--splice target_exons.tsv \
--reverse_complement \
--consensus \
-o rho_full_consensus

no populationmap provided


Target region: 3793413 - 3794477
writing haplotypes to fasta



Select subset of taxa for selection analysis.

In [5]:
!mkdir target_only_01

In [6]:
cd target_only_01/

/home/chrishah/WORKING/Diplotaxodon/per_gene/Rhodopsin_topology_s81/target_only_01


In [7]:
from Bio import AlignIO

ali = AlignIO.read(open("../rho_full_consensus.fas", 'r'),'fasta')

total_length = len(ali[0, :].seq)
print "Total length: %s\n" %total_length

sps = []

for rec in ali:
    sps.append(rec.id)
    
for sp in sorted(sps):
    print sp

Total length: 1065

A_calliptera_Bua
A_calliptera_Chitimba
A_calliptera_Chizumulu
A_calliptera_Enukweni
A_calliptera_Itupi_1
A_calliptera_Itupi_2
A_calliptera_Itupi_3
A_calliptera_Itupi_4
A_calliptera_Kitai_Dam
A_calliptera_Lake_Chidya
A_calliptera_Lake_Chilwa
A_calliptera_Luwawa
A_calliptera_Mbaka_river_Female
A_calliptera_Near_Kyela
A_calliptera_North_Rukuru
A_calliptera_Rovuma_river_Female
A_calliptera_Salima_Father
A_calliptera_Salima_Mother
A_calliptera_Salima_Offspring
A_calliptera_Songwe_River
A_calliptera_South_Rukuru
A_calliptera_Upper_Rovuma
Alticorpus_geoffreyi
Alticorpus_macrocleithrum
Astatotilapia_Kingiri
Astatotilapia_rujewa
Astatotilapia_tweddlei
Aulonocara_minutus
Aulonocara_steveni
Aulonocara_stuartgranti_Father
Aulonocara_stuartgranti_Mother
Aulonocara_stuartgranti_Offspring
Aulonocara_yellow
Buccochromis_nototaenia
Buccochromis_rhoadesii
Champsochromis_caeruelus_1
Champsochromis_caeruelus_2
Chilotilapia_rhoadesii_1
Chilotilapia_rhoadesii_2
Chilotilapia_rhoadesii_3
C

In [8]:
target_list = ['Alticorpus_geoffreyi',
               'Alticorpus_macrocleithrum',
               'Astatotilapia_tweddlei',
               'Aulonocara_minutus',
               'Aulonocara_steveni',
               'Aulonocara_yellow',
               'Buccochromis_nototaenia',
               'Buccochromis_rhoadesii',
               'Chilotilapia_rhoadesii_1',
               'Copadichromis_quadrimaculatus',
               'Copadichromis_trimaculatus',
               'Copadichromis_virginalis',
               'Ctenopharynx_nitidus',
               'Ctenopharynx_intermedius_1',
               'Cynotilapia_afra',
               'Cynotilapia_axelrodi',
               'Dimidiochromis_compressiceps',
               'Dimidiochromis_dimidiatus',
               'Dimidiochromis_strigatus',
               'Diplotaxodon_greenwoodi',
               'Diplotaxodon_limnothrissa',
               'Diplotaxodon_macrops',
               'Diplotaxodon_macrops_black_dorsal',
               'Diplotaxodon_ngulube',
               'Diplotaxodon_white_back_similis',
               'Iodotropheus_sprengerae',
               'Labeotropheus_trewavasae',
               'Lethrinops_albus',
               'Lethrinops_auritus',
               'Lethrinops_lethrinus',
               'Lethrinops_gossei',
               'Lethrinops_longimanus_redhead',
               'Lethrinops_sp_oliveri',
               'Metriaclima_zebra',
               'Mylochromis_anaphyrmus',
               'Nimbochromis_linni',
               'Nimbochromis_livingstoni',
               'Nimbochromis_polystigma',
               'Otopharynx_lithobates',
               'Otopharynx_speciosus_1',
               'Pallidochromis_tokolosh',
               'Genyochromis_mento',
               'Petrotilapia_genalutea',
               'Placidochromis_johnstoni',
               'Placidochromis_longimanus_1',
               'Stigmatochromis_guttatus',
               'Tropheops_tropheops',
               'Taeniolethrinops_furcicauda',
               'Tyrannochromis_nigriventer']

Reduce alignment to only target taxa.

In [9]:
from Bio import AlignIO
from Bio.Alphabet import generic_dna
from Bio.Align import MultipleSeqAlignment

reduced_ali = MultipleSeqAlignment([],alphabet=generic_dna)

for rec in ali:
    if rec.id in target_list:
            reduced_ali.append(rec)

AlignIO.write(reduced_ali[:,:], 'rho_full_consensus_target_only_01.fas', 'fasta')

1

Convert Alignment to Nexus.

In [10]:
AlignIO.write(reduced_ali[:,:], 'rho_full_consensus_target_only_01.nex', 'nexus')

1

PAML has an easier time in reading sequential NEXUS format so I'll reformat it from interleaved to sequential. Interestingly for short alignments it seems to result in an sequential NEXUS file already with the above command. For a long alignment it became interleaved so the below cell makes sure we've got sequential NEXUS.

In [11]:
from Bio import Nexus

Nexus.Nexus.Nexus.read

aln = Nexus.Nexus.Nexus()

aln.read('rho_full_consensus_target_only_01.nex')

aln.write_nexus_data('rho_full_consensus_target_only_01_sequential.nex', 
                     interleave=False, mrbayes=True)

'rho_full_consensus_target_only_01_sequential.nex'

Add tree block to Nexus file.

In [12]:
raxml_tree = open('../../../Phylogeny/s81/target_only_01/RAxML_bipartitions.s81_consensus_target_only_1.nobranchlenghts.nwk', 'r')

tree = raxml_tree.next().strip()

nex_file = open('rho_full_consensus_target_only_01.nex', 'a')

nex_file.write('begin trees;\n\ttree tree1 = %s\nend;' %tree)

nex_file.close()

Add tree with branchlenghts.

In [13]:
AlignIO.write(reduced_ali[:,:], 'rho_full_consensus_target_only_01.branchlengths.nex', 'nexus')

raxml_tree = open('../../../Phylogeny/s81/target_only_01/RAxML_bipartitions.s81_consensus_target_only_1.rooted_branchlenghts.nwk', 'r')

tree = raxml_tree.next().strip()

nex_file = open('rho_full_consensus_target_only_01.branchlengths.nex', 'a')

nex_file.write('begin trees;\n\ttree tree1 = %s\nend;' %tree)

nex_file.close()

Submit to [datamonkey.org](http://datamonkey.org/) for tests of selection.

## Run PAML.

In [14]:
mkdir PAML

In [15]:
cd PAML/

/home/chrishah/WORKING/Diplotaxodon/per_gene/Rhodopsin_topology_s81/target_only_01/PAML


In [16]:
%%bash

cp ../../../../Phylogeny/s81/target_only_01/RAxML_bipartitions.s81_consensus_target_only_1.nobranchlenghts.nwk .
cp ../../../../Phylogeny/s81/target_only_01/RAxML_bipartitions.s81_consensus_target_only_1.rooted_branchlenghts.nwk .

cp ../rho_full_consensus_target_only_01_sequential.nex .

The M0 model uses a single omega value across all sites in the alignment, and estimates average omega across the entire alignment.

In [17]:
mkdir M0

In [40]:
cd M0/

/home/chrishah/WORKING/Diplotaxodon/per_gene/Rhodopsin_topology_s81/target_only_01/PAML/M0


In [19]:
!cp ../codeml.template.ctl codeml.ctl

Modify codeml control file.

In [20]:
!cat codeml.ctl

       seqfile = ../rho_full_consensus_target_only_01_sequential.nex        * sequence data filename

     treefile = ../RAxML_bipartitions.s81_consensus_target_only_1.nobranchlenghts.nwk      * SET THIS for tree file with ML branch lengths under M0

      outfile = results.txt           * main result file name
        noisy = 9                     * lots of rubbish on the screen
      verbose = 1                     * detailed output
      runmode = 0                     * user defined tree
      seqtype = 1                     * codons
    CodonFreq = 2                     * F3X4 for codon ferquencies
        model = 0                     * one omega ratio for all branches




    * NSsites = 3                     * SET THIS for M3

    * NSsites = 8                     * SET THIS for M8

        icode = 0                     * universal code
    fix_kappa = 0                     * kappa fixed
       kappa = 2               	      * SET THIS to fix kappa at MLE under M0

    fix_omeg

Run PAML.

In [21]:
!~/src/PAML/paml4.8/bin/codeml codeml.ctl


 15         verbose | verbose                1.00
  7         runmode | runmode                0.00
  4         seqtype | seqtype                1.00
 13       CodonFreq | CodonFreq              2.00
 16           model | model                  0.00
 20         NSsites | NSsites                0.00
 22           icode | icode                  0.00
 24       fix_kappa | fix_kappa              0.00
 25           kappa | kappa                  2.00
 26       fix_omega | fix_omega              0.00
 27           omega | omega                  0.50
 37     fix_blength | fix_blength            0.00
  6       cleandata | cleandata              0.00
CODONML in paml version 4.8a, August 2014

----------------------------------------------
Phe F TTT | Ser S TCT | Tyr Y TAT | Cys C TGT
      TTC |       TCC |       TAC |       TGC
Leu L TTA |       TCA | *** * TAA | *** * TGA
      TTG |       TCG |       TAG | Trp W TGG
----------------------------------------------
Leu L CTT | Pro P CCT | His 

File format issues (see above). Insert second space after D. macrops black dorsal.

In [21]:
!sed -i 's/Diplotaxodon_macrops_black_dorsal /Diplotaxodon_macrops_black_dorsal  /' ../rho_full_consensus_target_only_01_sequential.nex

Run PAML.

In [ ]:
!~/src/PAML/paml4.8/bin/codeml codeml.ctl #was actually run in the shell because the user was required to confirm with enter

Extract the log likelihood for model M0 from the result.

In [41]:
!cat results.txt | grep "lnL"

lnL(ntime: 96  np: 98):  -1957.968493      +0.000000


In [42]:
cd ..

/home/chrishah/WORKING/Diplotaxodon/per_gene/Rhodopsin_topology_s81/target_only_01/PAML


The M1a model allows for a fraction of sites with ω < 1 and a fraction with ω = 1.

In [24]:
mkdir M1a

In [43]:
cd M1a/

/home/chrishah/WORKING/Diplotaxodon/per_gene/Rhodopsin_topology_s81/target_only_01/PAML/M1a


In [26]:
!cp ../codeml.template.ctl codeml.ctl

In [27]:
!cat codeml.ctl

       seqfile = ../rho_full_consensus_target_only_01_sequential.nex        * sequence data filename

     treefile = ../RAxML_bipartitions.s81_consensus_target_only_1.nobranchlenghts.nwk      * SET THIS for tree file with ML branch lengths under M0

      outfile = results.txt           * main result file name
        noisy = 9                     * lots of rubbish on the screen
      verbose = 1                     * detailed output
      runmode = 0                     * user defined tree
      seqtype = 1                     * codons
    CodonFreq = 2                     * F3X4 for codon ferquencies
        model = 0                     * one omega ratio for all branches




    * NSsites = 3                     * SET THIS for M3

    * NSsites = 8                     * SET THIS for M8

        icode = 0                     * universal code
    fix_kappa = 0                     * kappa fixed
       kappa = 2               	      * SET THIS to fix kappa at MLE under M0

    fix_omeg

In [ ]:
!~/src/PAML/paml4.8/bin/codeml codeml.ctl #was actually run in the shell because the user was required to confirm with enter

Extract the log likelihood for model M1 from the result.

In [44]:
!cat results.txt | grep "lnL"

lnL(ntime: 96  np: 99):  -1925.774096      +0.000000


In [45]:
cd ..

/home/chrishah/WORKING/Diplotaxodon/per_gene/Rhodopsin_topology_s81/target_only_01/PAML


Model M0 is nested within model M1, so we can use the likelihood ratio test (LRT) to compare these two models. 

The LRT statistic is χ2 = 2×(lnLM1 - lnLM0). 

The degrees of freedom are k = npM1 - npM1. 

We can use the CHI2 program from the PAML package to assess whether the value χ2 at certain degrees of freedom k is statistically significant.

In [46]:
%%bash

H0_file='M0/results.txt'
H1_file='M1a/results.txt'

dfM0=$(cat $H0_file | grep "lnL" | perl -ne 'chomp; @a=split(" "); $out=$a[-3]; $out =~ s/\)\://; $out =~ s/np\://; print "$out\n"')
dfM1=$(cat $H1_file | grep "lnL" | perl -ne 'chomp; @a=split(" "); $out=$a[-3]; $out =~ s/\)\://; $out =~ s/np\://; print "$out\n"')

lnLM0=$(cat $H0_file | grep "lnL" | perl -ne 'chomp; @a=split(" "); $out=$a[4]; print "$out\n"')
lnLM1=$(cat $H1_file | grep "lnL" | perl -ne 'chomp; @a=split(" "); $out=$a[4]; print "$out\n"')

k=$(($dfM1-$dfM0))


dLRT=$(echo -e "$lnLM0 $lnLM1" | perl -ne 'chomp; @a=split(" "); $LRT=2*($a[1]-$a[0]); $out=sprintf("%.2f",$LRT); print "$out\n"')

echo -e "deltaLRT = $dLRT"
echo -e "degrees of freedom: $k"

/home/chrishah/src/PAML/paml4.8/bin/chi2 $k $dLRT

deltaLRT = 64.39
degrees of freedom: 1

df =  1  prob = 0.000000000 = 9.992e-16



The chi2 value from the above comparison is statstically significant at the significance level alpha=0.05. Therefore the inclusion of a class of neutral sites with ω = 1 is statistically justified and we can move on to test for positive selection.

Model M2a allows for a third class of sites with ω > 1.

In [31]:
mkdir M2a

In [47]:
cd M2a/

/home/chrishah/WORKING/Diplotaxodon/per_gene/Rhodopsin_topology_s81/target_only_01/PAML/M2a


In [33]:
cp ../codeml.template.ctl codeml.ctl

In [34]:
!cat codeml.ctl

       seqfile = ../rho_full_consensus_target_only_01_sequential.nex        * sequence data filename

     treefile = ../RAxML_bipartitions.s81_consensus_target_only_1.nobranchlenghts.nwk      * SET THIS for tree file with ML branch lengths under M0

      outfile = results.txt           * main result file name
        noisy = 9                     * lots of rubbish on the screen
      verbose = 1                     * detailed output
      runmode = 0                     * user defined tree
      seqtype = 1                     * codons
    CodonFreq = 2                     * F3X4 for codon ferquencies
        model = 0                     * one omega ratio for all branches




    * NSsites = 3                     * SET THIS for M3

    * NSsites = 8                     * SET THIS for M8

        icode = 0                     * universal code
    fix_kappa = 0                     * kappa fixed
       kappa = 2               	      * SET THIS to fix kappa at MLE under M0

    fix_omeg

In [ ]:
!~/src/PAML/paml4.8/bin/codeml codeml.ctl #was actually run in the shell because the user was required to confirm with enter

Extract the log likelihood for model M2 from the result.

In [49]:
!cat results.txt | grep "lnL"

lnL(ntime: 96  np:101):  -1804.732703      +0.000000


In [50]:
cd ..

/home/chrishah/WORKING/Diplotaxodon/per_gene/Rhodopsin_topology_s81/target_only_01/PAML


Model M1 is nested within model M2, so we can use the likelihood ratio test (LRT) to compare these two models. 

The LRT statistic is χ2 = 2×(lnLM2 - lnLM1). 

The degrees of freedom are k = npM2 - npM1. 

We can use the CHI2 program from the PAML package to assess whether the value χ2 at certain degrees of freedom k is statistically significant.

In [51]:
%%bash

H0_file='M1a/results.txt'
H1_file='M2a/results.txt'

dfM0=$(cat $H0_file | grep "lnL" | perl -ne 'chomp; @a=split(" "); $out=$a[-3]; $out =~ s/\)\://; $out =~ s/np\://; print "$out\n"')
dfM1=$(cat $H1_file | grep "lnL" | perl -ne 'chomp; @a=split(" "); $out=$a[-3]; $out =~ s/\)\://; $out =~ s/np\://; print "$out\n"')

lnLM0=$(cat $H0_file | grep "lnL" | perl -ne 'chomp; @a=split(" "); $out=$a[-2]; print "$out\n"')
lnLM1=$(cat $H1_file | grep "lnL" | perl -ne 'chomp; @a=split(" "); $out=$a[-2]; print "$out\n"')

k=$(($dfM1-$dfM0))
dLRT=$(echo -e "$lnLM0 $lnLM1" | perl -ne 'chomp; @a=split(" "); $LRT=2*($a[1]-$a[0]); $out=sprintf("%.2f",$LRT); print "$out\n"')

echo -e "deltaLRT = $dLRT"
echo -e "degrees of freedom: $k"

/home/chrishah/src/PAML/paml4.8/bin/chi2 $k $dLRT

deltaLRT = 242.08
degrees of freedom: 2

df =  2  prob = 0.000000000 = 0.000e+00



The test statistic is highly significant, so the model M2a fits the data better, so including a site class with omega > 1 is statistically justified, which means we've got a signature of positive selection.

CODEML will perform Bayesian identification of codon sites (columns in the alignment) that are under positive selection. We will look at the results from the Bayes empirical Bayes (BEB) method in the results file of the model M2a.

Bayesian identification of codon sites (columns in the
alignment) that are under positive selection

In [52]:
!cat M2a/results.txt | grep "BEB" -A 21

Bayes Empirical Bayes (BEB) analysis (Yang, Wong & Nielsen 2005. Mol. Biol. Evol. 22:1107-1118)
Positively selected sites (*: P>95%; **: P>99%)
(amino acids refer to 1st sequence: Nimbochromis_polystigma)

            Pr(w>1)     post mean +- SE for w

    22 S      1.000**       10.493 +- 0.125
    29 H      0.986*        10.361 +- 1.123
    37 Y      1.000**       10.493 +- 0.124
    39 A      0.986*        10.361 +- 1.123
    83 D      0.999**       10.482 +- 0.350
    95 V      0.999**       10.482 +- 0.342
   133 I      1.000**       10.494 +- 0.078
   166 A      1.000**       10.494 +- 0.078
   173 V      0.829         8.819 +- 3.686
   189 V      0.999**       10.481 +- 0.359
   213 I      1.000**       10.494 +- 0.078
   217 T      0.845         8.975 +- 3.542
   297 G      1.000**       10.494 +- 0.078
   298 A      1.000**       10.494 +- 0.079
   299 S      1.000**       10.494 +- 0.078

